In [6]:
from pulp import *
import random

class Pattern:
    def __init__(self, name, lengths=None, lenOpts=None, totalRollLength=20):
        self.name = name.replace(" ", "_")
        self.lenOpts = lenOpts or ["5", "7", "9"]
        self.totalRollLength = totalRollLength
        self.lengthsdict = dict(zip(self.lenOpts, lengths))

    def __str__(self):
        return f"Pattern(name={self.name}, lengths={self.lengthsdict})"

    def trim(self):
        return self.totalRollLength - sum(
            [int(i) * int(self.lengthsdict[i]) for i in self.lengthsdict]
        )

def generateExample(num_lengths=3, max_demand=100, totalRollLength=50):
    """
    Generates input data for the Cutting Stock problem.
    """
    # Generate possible roll lengths
    lengths = [str(random.randint(3, 10)) for _ in range(num_lengths)]

    # Generate roll demand
    rollData = {length: (random.randint(1, max_demand), random.uniform(0.1, 1.0)) for length in lengths}

    # Generate initial patterns
    Patterns = []
    for i, length in enumerate(lengths):
        pattern_lengths = [0] * len(lengths)
        pattern_lengths[i] = totalRollLength // int(length)
        Patterns.append(Pattern(f"P{i+1}", pattern_lengths, lenOpts=lengths, totalRollLength=totalRollLength))

    return rollData, Patterns, lengths, totalRollLength

def masterSolve(Patterns, rollData, lenOpts, totalRollLength, relax=True, iteration=0):
    """
    Solves the master problem of the Column Generation method.
    """
    global constraint_counter
    rollDemand = {key: val[0] for key, val in rollData.items()}

    # Create the optimization problem
    prob = LpProblem("Cutting_Stock_Problem", LpMinimize)
    vartype = LpContinuous if relax else LpInteger
    pattVars = LpVariable.dicts("Pattern", Patterns, 0, None, vartype)

    # Objective: minimize the number of large rolls used
    prob += lpSum([pattVars[i] * 1 for i in Patterns])

    # Constraints: satisfy demand for each roll length
    for j in lenOpts:
        constraint_name = f"Min{j}_Iter{iteration}_{constraint_counter}"  # Ensure absolute uniqueness
        constraint_counter += 1
        prob += (
            lpSum([pattVars[i] * i.lengthsdict[j] for i in Patterns])
            >= rollDemand[j],
            constraint_name,
        )

    prob.solve()

    if relax:
        duals = {
            j: prob.constraints[f"Min{j}_Iter{iteration}_{constraint_counter - len(lenOpts) + idx}"].pi
            for idx, j in enumerate(lenOpts)
        }
        return duals
    else:
        varsdict = {v.name: v.varValue for v in prob.variables()}
        return value(prob.objective), varsdict

def subSolve(Patterns, duals, lenOpts, totalRollLength):
    """
    Solves the subproblem of the Column Generation method.
    """
    # Create the optimization problem
    prob = LpProblem("SubProb", LpMinimize)
    _vars = LpVariable.dicts("Roll Length", lenOpts, 0, None, LpInteger)
    trim = LpVariable("Trim", 0, None, LpInteger)

    # Objective: minimize the cost of a new pattern
    prob += (1 - lpSum([_vars[i] * duals[i] for i in lenOpts])), "Objective"

    # Constraint: ensure the total length of the new roll matches the allowed length
    prob += (
        lpSum([_vars[i] * int(i) for i in lenOpts]) + trim
        == totalRollLength,
        "lengthEquate",
    )

    # Save the coefficients, matrix, and RHS (right-hand side) in LP history
    matrix = [[int(i) for i in lenOpts]]
    rhs = [totalRollLength]
    coefficients = {i: duals[i] for i in lenOpts}
    lp_history.append({"coefficients": coefficients, "matrix": matrix, "rhs": rhs})

    prob.solve()

    # Generate a new pattern based on the solution
    newPattern = {i: int(_vars[i].varValue) for i in lenOpts}
    if value(prob.objective) < -(10**-5):
        morePatterns = True
        if newPattern not in [p.lengthsdict for p in Patterns]:
            Patterns.append(Pattern(f"P{len(Patterns) + 1}", [newPattern[i] for i in lenOpts], lenOpts, totalRollLength))
    else:
        morePatterns = False

    return Patterns, morePatterns


def columnGeneration(Patterns, rollData, lenOpts, totalRollLength):
    """
    Executes the iterative Column Generation process.
    """
    iterations = 0
    while True:
        # Solve the master problem
        duals = masterSolve(Patterns, rollData, lenOpts, totalRollLength, relax=True)

        # Solve the subproblem
        Patterns, morePatterns = subSolve(Patterns, duals, lenOpts, totalRollLength)

        # If no new patterns are generated, terminate
        if not morePatterns:
            break
        iterations += 1

    return Patterns


# Global variable to store LP history for the subproblem (coefficients, matrix, and right-hand side)
lp_history = []

# Global counter for unique constraint names
constraint_counter = 0

# Generate random input data
rollData, Patterns, lenOpts, totalRollLength = generateExample()

print("num of initial patterns: ", len(Patterns))
print(Patterns)
# Solve the problem using Column Generation
optimalPatterns = columnGeneration(Patterns, rollData, lenOpts, totalRollLength)
print("num of optimal patterns: ", len(optimalPatterns))
# Display results
print("\nGenerated Roll Data:")
for length, (demand, _) in rollData.items():
    print(f"Length: {length}, Demand: {demand}")

print("\nOptimal Patterns:")
for pattern in optimalPatterns:
    print(pattern)

# Print the LP history for the subproblem (coefficients, matrix, RHS)
print("\nSubproblem LP History:")
for i, lp in enumerate(lp_history, 1):
    print(f"--- Subproblem Iteration {i} ---")
    print(f"Coefficients: {lp['coefficients']}")
    print(f"Matrix: {lp['matrix']}")
    print(f"RHS: {lp['rhs']}")

num of initial patterns:  3
[<__main__.Pattern object at 0x7294a4ff8f10>, <__main__.Pattern object at 0x7294a4ffbb80>, <__main__.Pattern object at 0x7294a4ff9120>]
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/jan/miniconda3/envs/ipmgnn/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/2a48bf59a8d148efb865ea97c93612ec-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/2a48bf59a8d148efb865ea97c93612ec-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 8 COLUMNS
At line 15 RHS
At line 19 BOUNDS
At line 20 ENDATA
Problem MODEL has 3 rows, 3 columns and 3 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-3) rows, 0 (-3) columns and 0 (-3) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 18.571429
After Postsolve, objective 18.571429, infeasibilities - dual 0 (0), primal 0 (0)
Optimal o

In [8]:
lp_history

[{'coefficients': {'4': 0.083333333, '8': 0.16666667, '7': 0.14285714},
  'matrix': [[4, 8, 7]],
  'rhs': [50]},
 {'coefficients': {'4': 0.083333333, '8': 0.16666667, '7': 0.125},
  'matrix': [[4, 8, 7]],
  'rhs': [50]}]

In [7]:
len(optimalPatterns)

4

In [5]:
len(Patterns)

5